In [1]:
from math import sqrt

class Grid_5x5:
    def __init__(self):
        self.grid = [ 
                      [3, 4, 1, 3, 1],
                      [3, 3, 3,'G',2],
                      [3, 1, 2, 2, 3],
                      [4, 2, 3, 3, 3],
                      [4, 1, 4, 3, 2]
                                        ]
        self.goal = 'G'
        self.goal_pos = {"row":1, "col":3}
    
    # =====================================================================================================================
    # Prints 5x5 Grid and also can bold and underline Agent's current state while printing Grid
    def print_environment(self, current_state=None):
        
        for r in range(5):
            for c in range(5):

                if current_state:
                    if r == current_state['row'] and c == current_state['col']:
                        # \033[1m is for bold, \033[4m is for underlined, \033[0m is for finishing both bold and underlined (all)
                        print("\033[1m\033[4m{}\033[0m".format(self.grid[r][c]), end=' ')
                    else:
                        print(self.grid[r][c], end=' ')
                
                else:
                    print(self.grid[r][c], end=' ')
                
            print()
        print()    
        return
    
    # =====================================================================================================================    
    # Gives the new row after adding any num to it or Gives thhe new column after adding any num to it, 
    def _increment_pos(self, row_or_col, num_to_move):
        return (row_or_col+num_to_move)%5   # If adding num_to_move to row_or_col exceeds 5 (given rows,cols of grid) so thats why using modulo to move in circular

In [2]:
# =====================================================================================================================
# Would be used in searching tree we construct later
class Node:
    def __init__(self, parent, state, parent_action, path_cost):
        self.parent = parent
        self.state = state
        self.parent_action = parent_action
        self.path_cost = path_cost 

# =====================================================================================================================        
class ChildNode(Node):
    def __init__(self, problem, parent, parent_action):
        state = problem.transition_model(parent.state, parent_action)  # This will give new state when a state applies an action
        path_cost = parent.path_cost + problem.step_cost(parent.state, parent_action)  # This would sum of step costs of path at each individual state

        super().__init__(parent=parent, 
                         state=state, 
                         parent_action=parent_action,
                         path_cost=path_cost)


In [3]:
# =====================================================================================================================
# Would be used in informed searching tree we construct later
class InformedNode:
    def __init__(self, parent, state, parent_action, path_cost, heuristic_score):
        self.parent = parent
        self.state = state
        self.parent_action = parent_action
        self.path_cost = path_cost   # g(n)
        self.heuristic_score = heuristic_score  # h(n)

# =====================================================================================================================        
class InformedChildNode(InformedNode):
    def __init__(self, problem, parent, parent_action, heuristic_type):
        state = problem.transition_model(parent.state, parent_action)  # This will give new state when a state applies an action
        path_cost = parent.path_cost + problem.step_cost(parent.state, parent_action)  # This would sum of step costs of path at each individual state
        heuristic_score = problem.calculate_heuristic(state, heuristic_type)  # calculating heuristic
#         print(parent.heuristic_score, heuristic_score)
        
        super().__init__(parent=parent, 
                         state=state, 
                         parent_action=parent_action,
                         path_cost=path_cost,
                         heuristic_score=heuristic_score)


In [4]:
# =====================================================================================================================
# According to Book a problem for searching has:
# 1. initial state
# 2. possible actions
# 3. transition model (A description what each action does)
# 4. goal test (which determines that has goal been reached at given state)
# 5. path cost (that assigns numeric cost to each path)
class Problem:
    def __init__(self, Environment, initial_state):
        self.initial_state = initial_state
        self.Environment = Environment
        self.possible_actions = ['horizontal', 'vertical']

    # =====================================================================================================================        
    # Gives new state given current state and action applied at current state
    def transition_model(self, current_state, action):
        state, new_state = current_state.copy(), current_state.copy()
        # Note: state/position in grid seemed better to represent as dictionary for readibility
        row = state['row']
        col = state['col']
        num_to_move = self.Environment.grid[row][col]
        
        # if action is to move horizontal then increment the current col of state according to current state's value
        if action == 'horizontal':
            new_state['col'] = self.Environment._increment_pos(col, num_to_move)
        
        # if action is to move vertical then increment the current row of state according to current state's value
        elif action == 'vertical':
            new_state['row'] = self.Environment._increment_pos(row, num_to_move)
            
        return new_state

    # =====================================================================================================================    
    # Tests that whether current node is goal state or not
    def goal_test(self, current_node):
        state = current_node.state
        row = state['row']
        col = state['col']
        value_in_grid = self.Environment.grid[row][col]
        
#         print('{},{} -> {}'.format(row, col, value_in_grid))
        if value_in_grid == self.Environment.goal:
            return True
        
        return False

    # =====================================================================================================================    
    # step cost of each individual step/state, as there are only two actions horizontally and vertically so 1 as step cost for both seems better
    def step_cost(self, current_state, action):
        return 1   # In book assumption is that step costs are non negative
    
    # =====================================================================================================================
    # calculate euclidean heuristic
    def euclidean_heuristic(self, state):
        goal_pos = self.Environment.goal_pos
        return  sqrt( (state['row']-goal_pos['row'])**2 
                                    +
                      (state['col']-goal_pos['col'])**2
                                                             )
    # =====================================================================================================================    
    # calculate manhattan heuristic
    def manhattan_heuristic(self, state):
        goal_pos = self.Environment.goal_pos
        return  ( abs( state['row']-goal_pos['row'] )
                                +
                  abs( state['col']-goal_pos['col'] )  )
            
    # =====================================================================================================================
    # calculate euclidean heuristic or manhattan heuristic of a state
    def calculate_heuristic(self, state, heuristic_type):
        if heuristic_type == 'euclidean':
            return self.euclidean_heuristic(state)
        
        elif heuristic_type == 'manhattan':
            return self.manhattan_heuristic(state)

In [5]:
# =====================================================================================================================
class GridSearchingAgent():
    def __init__(self, Problem):
#         Problem.Environment.print_environment()
#         Problem.Environment.print_environment(Problem.initial_state)
        
        self.Environment = Problem.Environment  # seems better
        self.Problem = Problem  

    # =====================================================================================================================    
    # Gives sequences of actions from from the branch where goal state was passed on leaf starting from parent state to leaf node (goal state)
    def actions_to_take(self, current_node):        
        if current_node.parent is None:   # base case for recursion
            return []
        
        return self.actions_to_take(current_node.parent) + [current_node.parent_action]

    # =====================================================================================================================
    # breadth first search algorithm, returns a sequences of action and performance measure
    def breadth_first_search_goal(self):
        node = Node(parent=None,
                    state=self.Problem.initial_state,
                    parent_action=None,
                    path_cost=0)
        search_cost = 1
        
        # If in Root node goal state has been achieved then return all sequences of actions        
        if self.Problem.goal_test(node):
            return self.actions_to_take(node), search_cost, node.path_cost
        
        frontier = [node]   # queue: Frontier is the set of all leaf nodes available for expansion at any given point is called the frontier according to book

        while len(frontier) != 0:
            current_node = frontier.pop(0)  # FIFO

            # Creating a child node for all possible actions of current state and checking goal state on each node
            for action in self.Problem.possible_actions:
                child = ChildNode(self.Problem, current_node, action)
                search_cost += 1
                
                if self.Problem.goal_test(child) is True:  # checking goal test
                    return self.actions_to_take(child), search_cost, child.path_cost  # if goal state return sequence of actions
                
                frontier.append(child)
                
        return None, None, None
    
    
    # =====================================================================================================================    
    # greedy best first search algorithm, returns a sequences of action and performance measure
    def greedy_best_first_search_goal(self, heuristic_type):
        node = InformedNode(parent=None,
                            state=self.Problem.initial_state,
                            parent_action=None,
                            path_cost=0,
                            heuristic_score=problem.calculate_heuristic(self.Problem.initial_state, heuristic_type))
        search_cost = 1
        
        # If in Root node goal state has been achieved then return all sequences of actions        
        if self.Problem.goal_test(node):
            return self.actions_to_take(node), search_cost, node.path_cost
        
        priority_queue = [node]   # Priority Queue on h(n) i.e heuristic of nodes

        while len(priority_queue) != 0:
            current_node = priority_queue.pop(0)  # FIFO

            # Creating a child node for all possible actions of current state and checking goal state on each node
            for action in self.Problem.possible_actions:
                child = InformedChildNode(self.Problem, current_node, action, heuristic_type)
                search_cost += 1
#                 print(child.parent.state, child.state)
                if self.Problem.goal_test(child) is True:  # checking goal test
                    return self.actions_to_take(child), search_cost, child.path_cost  # if goal state return sequence of actions
                
                priority_queue.append(child)
            
            # f(n) = h(n)  for priority queue
            priority_queue = sorted(priority_queue, key=lambda node: node.heuristic_score)
            
        return None, None, None    

    
    # =====================================================================================================================    
    # A* search algorithm, returns a sequences of action and performance measure
    def astar_search_goal(self, heuristic_type):
        node = InformedNode(parent=None,
                            state=self.Problem.initial_state,
                            parent_action=None,
                            path_cost=0,
                            heuristic_score=problem.calculate_heuristic(self.Problem.initial_state, heuristic_type))
        search_cost = 1
        
        # If in Root node goal state has been achieved then return all sequences of actions        
        if self.Problem.goal_test(node):
            return self.actions_to_take(node), search_cost, node.path_cost
        
        priority_queue = [node]   # Priority Queue on f(n)=g(n)+h(n) of nodes where g(n) is path cost from start node and h(n) is heuristic
#         print(node.state)

        while len(priority_queue) != 0:
            current_node = priority_queue.pop(0)  # FIFO

            # Creating a child node for all possible actions of current state and checking goal state on each node
            for action in self.Problem.possible_actions:
                child = InformedChildNode(self.Problem, current_node, action, heuristic_type)
                search_cost += 1
                
                if self.Problem.goal_test(child) is True:  # checking goal test
                    return self.actions_to_take(child), search_cost, child.path_cost  # if goal state return sequence of actions
                
                priority_queue.append(child)
            
            # f(n) = g(n)+h(n)  for priority queue where g(n) is cost from start node and h(n) is heuristic
            priority_queue = sorted(priority_queue, key=lambda node: node.path_cost + node.heuristic_score)
            
        return None, None, None    
    
    
    # =====================================================================================================================
    # This helper method turns a state {row:x col:y} to (x,y) Note: state/position in grid seemed better to represent as dictionary for readibility but for displaying tuple seemed better
    def _state_to_tuple(self, state):
        x = state['row']
        y = state['col']
        return x,y
    
    # =====================================================================================================================
    # This helper method gives new state (used for printing)
    def _change_state(self, state, action):
        return self.Problem.transition_model(state, action)
    
    # =====================================================================================================================
    # This helper method displays
    def display_action(self, current_state, action):
        current_pos = self._state_to_tuple(current_state)
        new_state = self._change_state(current_state, action)
        new_pos = self._state_to_tuple(new_state)
        
        print('Agent moving {} from {} to {}'.format(action, current_pos, new_pos))
        self.Environment.print_environment(new_state)
              
        return new_state
    
    # =====================================================================================================================
    # This method will do searching and if solution exists it will also display the actions          
    def start(self, search_algo, heuristic_type=None):
        print("\n====== {} with h(n)={}========".format(search_algo.upper(), heuristic_type))
        print("\n---Agent's initial state is {}---".format( self._state_to_tuple(self.Problem.initial_state) ) )
        self.Problem.Environment.print_environment(self.Problem.initial_state)
                      
        current_state = self.Problem.initial_state
        
        # searching for solution
        if search_algo == 'bfs':
            solution = self.breadth_first_search_goal()
        elif search_algo == 'greedy':
            solution = self.greedy_best_first_search_goal(heuristic_type)
        elif search_algo == 'A*':
            solution = self.astar_search_goal(heuristic_type)
            
        actions_sequence, search_cost, path_cost = solution
        
        if actions_sequence:
            for action in actions_sequence:
                current_state = self.display_action(current_state, action)
                
        print("---Agent has reached 'G' so stopping")
        self.Environment.print_environment(current_state)
        print("search cost:", search_cost)
        print("path cost:", path_cost)
        print("total cost:", search_cost+path_cost)  # total cost combines both search cost and path cost
        print('\n')
        
        return

In [6]:
environment = Grid_5x5()
initial_state = {'row':0, 'col':0}

problem = Problem(environment, initial_state)
agent1 = GridSearchingAgent(problem)
agent2 = GridSearchingAgent(problem)
agent3 = GridSearchingAgent(problem)

agent1.start('bfs')
agent2.start('greedy', 'euclidean')
agent3.start('A*', 'euclidean')


====== BFS with h(n)=None========

---Agent's initial state is (0, 0)---
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

Agent moving horizontal from (0, 0) to (0, 3)
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

Agent moving vertical from (0, 3) to (3, 3)
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

Agent moving vertical from (3, 3) to (1, 3)
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

---Agent has reached 'G' so stopping
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

search cost: 11
path cost: 3
total cost: 14



====== GREEDY with h(n)=euclidean========

---Agent's initial state is (0, 0)---
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

Agent moving horizontal from (0, 0) to (0, 3)
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

Agent moving vertical from (0, 3) to (3, 3)
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

Agent moving vertical from (3, 3) to (1, 3)
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4

In [7]:
environment = Grid_5x5()
initial_state = {'row':0, 'col':0}

problem = Problem(environment, initial_state)
agent1 = GridSearchingAgent(problem)
agent2 = GridSearchingAgent(problem)
agent3 = GridSearchingAgent(problem)

agent1.start('bfs')
agent2.start('greedy', 'manhattan')
agent3.start('A*', 'manhattan')


====== BFS with h(n)=None========

---Agent's initial state is (0, 0)---
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

Agent moving horizontal from (0, 0) to (0, 3)
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

Agent moving vertical from (0, 3) to (3, 3)
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

Agent moving vertical from (3, 3) to (1, 3)
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

---Agent has reached 'G' so stopping
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

search cost: 11
path cost: 3
total cost: 14



====== GREEDY with h(n)=manhattan========

---Agent's initial state is (0, 0)---
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

Agent moving horizontal from (0, 0) to (0, 3)
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

Agent moving vertical from (0, 3) to (3, 3)
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

Agent moving vertical from (3, 3) to (1, 3)
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4

In [8]:
for i in range(0,5):
    for j in range(0,5):
        print('...... initial state {} {}'.format(i,j))
        environment = Grid_5x5()
        initial_state = {'row':i, 'col':j}

        problem = Problem(environment, initial_state)
        agent1 = GridSearchingAgent(problem)
        agent2 = GridSearchingAgent(problem)
        agent3 = GridSearchingAgent(problem)

        agent1.start('bfs')
#         print('===== GREEDY EUCLIDEAN HEURISTIC ======')   # Goes to infinity
#         agent2.start('greedy', 'euclidean')
        print('===== A* EUCLIDEAN HEURISTIC ======')
        agent3.start('A*', 'euclidean')        

...... initial state 0 0

====== BFS with h(n)=None========

---Agent's initial state is (0, 0)---
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

Agent moving horizontal from (0, 0) to (0, 3)
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

Agent moving vertical from (0, 3) to (3, 3)
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

Agent moving vertical from (3, 3) to (1, 3)
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

---Agent has reached 'G' so stopping
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

search cost: 11
path cost: 3
total cost: 14


===== A* EUCLIDEAN HEURISTIC ======

====== A* with h(n)=euclidean========

---Agent's initial state is (0, 0)---
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

Agent moving horizontal from (0, 0) to (0, 3)
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

Agent moving vertical from (0, 3) to (3, 3)
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

Agent moving vertical from (3, 3

3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

Agent moving vertical from (3, 2) to (1, 2)
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

Agent moving horizontal from (1, 2) to (1, 0)
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

Agent moving horizontal from (1, 0) to (1, 3)
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

---Agent has reached 'G' so stopping
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

search cost: 56
path cost: 5
total cost: 61


...... initial state 2 0

====== BFS with h(n)=None========

---Agent's initial state is (2, 0)---
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

Agent moving vertical from (2, 0) to (0, 0)
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

Agent moving horizontal from (0, 0) to (0, 3)
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

Agent moving vertical from (0, 3) to (3, 3)
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

Agent moving vertical from (3, 3) to (1, 3)
3 4 1 3 1 
3 3 3 G 2 


search cost: 3
path cost: 1
total cost: 4


...... initial state 3 4

====== BFS with h(n)=None========

---Agent's initial state is (3, 4)---
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

Agent moving horizontal from (3, 4) to (3, 2)
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

Agent moving vertical from (3, 2) to (1, 2)
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

Agent moving horizontal from (1, 2) to (1, 0)
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

Agent moving horizontal from (1, 0) to (1, 3)
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

---Agent has reached 'G' so stopping
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

search cost: 20
path cost: 4
total cost: 24


===== A* EUCLIDEAN HEURISTIC ======

====== A* with h(n)=euclidean========

---Agent's initial state is (3, 4)---
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3 3 
4 1 4 3 2 

Agent moving horizontal from (3, 4) to (3, 2)
3 4 1 3 1 
3 3 3 G 2 
3 1 2 2 3 
4 2 3 3

In [ ]:
environment = Grid_5x5()
row_input = int(input("Enter the ROW of initial state in 5x5 grid: "))
col_input = int(input("Enter the COL of initial state in 5x5 grid: "))
initial_state = {'row':row_input, 'col':col_input}

problem = Problem(environment, initial_state)
agent = GridSearchingAgent(problem)

search_algo = input("Enter the search algorithm (bfs, greedy, A*): ")
heuristic_type = input("Enter the heuristic (euclidean, manhattan, None for bfs): ")
agent.start(search_algo, heuristic_type)